In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install opencv-python-headless

In [ ]:
!pip install pydot
!pip install graphviz

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

In [ ]:
# Paths to directories containing the images
lumpy_dir_path = '/content/drive/MyDrive/Lumpy_Skin_Disease/Dataset/Lumpy Skin'  # Replace with your actual path
healthy_dir_path = '/content/drive/MyDrive/Lumpy_Skin_Disease/Dataset/Normal Skin'  # Replace with your actual path

# Function to load images and labels into a DataFrame
def load_images_labels(dir_path, label):
    """Load image paths and labels from the specified directory."""
    images = []
    for filename in os.listdir(dir_path):
        if filename.endswith((".png", ".jpg", ".jpeg")):  # Add or modify the file types as needed
            image_path = os.path.join(dir_path, filename)
            images.append({'image_path': image_path, 'label': label})
    return pd.DataFrame(images)

# Load data from directories
lumpy_df = load_images_labels(lumpy_dir_path, 'Lumpy')
healthy_df = load_images_labels(healthy_dir_path, 'Healthy')

# Concatenate the two DataFrames into one
combined_df = pd.concat([lumpy_df, healthy_df]).reset_index(drop=True)

print(combined_df)
print(lumpy_df)
print(healthy_df)


                                             image_path    label
0     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy
1     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy
2     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy
3     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy
4     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy
...                                                 ...      ...
1138  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy
1139  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy
1140  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy
1141  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy
1142  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy

[1143 rows x 2 columns]
                                            image_path  label
0    /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Lumpy
1    /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Lumpy
2    /con

In [ ]:
# Function to read and preprocess an image
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        # Adjust this section based on your preprocessing requirements:
        # Cropping, resizing, normalization, etc.
        image_resized = cv2.resize(image, (224, 224))  # Resize to (224, 224) for example
        return image_resized
    else:
        return None  # If the image doesn't load correctly

# Applying the preprocessing function to all image paths
combined_df['preprocessed_image'] = combined_df['image_path'].apply(load_and_preprocess_image)

print(combined_df)

                                             image_path    label  \
0     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy   
1     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy   
2     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy   
3     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy   
4     /content/drive/MyDrive/Lumpy_Skin_Disease/Data...    Lumpy   
...                                                 ...      ...   
1138  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy   
1139  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy   
1140  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy   
1141  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy   
1142  /content/drive/MyDrive/Lumpy_Skin_Disease/Data...  Healthy   

                                     preprocessed_image  
0     [[[235, 251, 239], [210, 234, 217], [218, 242,...  
1     [[[246, 221, 197], [227, 203, 179], [239, 220,...  
2     [[[

In [ ]:
#Converting the images to numpy arrays and extracting the labels
images = np.array(combined_df['preprocessed_image'].tolist())
labels = combined_df['label'].map({'Lumpy': 1, 'Healthy': 0}).values

In [ ]:
# Spliting the dataset into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Defining the CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Converting data to a format suitable for training
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Using ImageDataGenerator flow method for the training set
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

# Using ImageDataGenerator flow method for the validation set
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

# Training the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)
model.summary()
# Saving the model
model.save('/content/drive/MyDrive/Lumpy_Skin_Disease/model.h5')

# Evaluating the model
evaluation = model.evaluate(val_generator)
print(f"Validation Loss: {evaluation[0]}, Validation Accuracy: {evaluation[1]}")

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


Epoch 1/50
29/29 [==============================] - 75s 3s/step - loss: 8.8219 - accuracy: 0.5733 - val_loss: 1.2200 - val_accuracy: 0.4367
Epoch 2/50
29/29 [==============================] - 69s 2s/step - loss: 0.5799 - accuracy: 0.7177 - val_loss: 0.4629 - val_accuracy: 0.7948
Epoch 3/50
29/29 [==============================] - 68s 2s/step - loss: 0.3178 - accuracy: 0.8862 - val_loss: 0.4259 - val_accuracy: 0.8734
Epoch 4/50
29/29 [==============================] - 79s 3s/step - loss: 0.1710 - accuracy: 0.9628 - val_loss: 0.4648 - val_accuracy: 0.8384
Epoch 5/50
29/29 [==============================] - 71s 2s/step - loss: 0.1056 - accuracy: 0.9781 - val_loss: 0.4849 - val_accuracy: 0.8428
Epoch 6/50
29/29 [==============================] - 75s 3s/step - loss: 0.0697 - accuracy: 0.9902 - val_loss: 0.4686 - val_accuracy: 0.8559
Epoch 7/50
29/29 [==============================] - 71s 2s/step - loss: 0.0444 - accuracy: 0.9934 - val_loss: 0.5792 - val_accuracy: 0.8384
Epoch 8/50
29/29 [==

In [ ]:
  from sklearn.metrics import confusion_matrix
  import matplotlib.pyplot as plt
  import seaborn as sns

  # Assuming val_generator from your provided code is used for validation
  # Predicting labels for all images in the validation set
  predictions = model.predict(val_generator)
  predicted_labels = (predictions > 0.5).astype(int)

  # Get true labels from the validation generator
  true_labels = y_val

  # Computing the confusion matrix
  cm = confusion_matrix(true_labels, predicted_labels.flatten())

  # Printing the confusion matrix
  print("Confusion Matrix:")
  print(cm)

  # Optionally, visualize the confusion matrix
  plt.figure(figsize=(4,4))
  sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=['Healthy', 'Lumpy'], yticklabels=['Healthy', 'Lumpy'])
  plt.xlabel('Predicted Labels')
  plt.ylabel('True Labels')
  plt.title('Confusion Matrix')
  plt.show()


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Loading the model
model_path = '/content/drive/MyDrive/Lumpy_Skin_Disease/model.h5'
model = load_model(model_path)

def predict_disease(image_path):
    # Loading the image file
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

    # Preprocessing the image - Ensuring this matches the preprocessing used in training
    img_preprocessed = cv2.resize(img, (224, 224))  # Resizing the image to 224x224 pixels
    img_preprocessed = img_preprocessed / 255.0  # Normalizing pixel values to [0, 1]
    img_preprocessed = np.expand_dims(img_preprocessed, axis=0)

    # Predicting the class of the image
    prediction = model.predict(img_preprocessed)

    # Determine the predicted class
    predicted_class = 'Lumpy' if prediction[0][0] > 0.5 else 'Healthy'

    # Display the image and the prediction
    plt.imshow(img_rgb)
    plt.title(f'Prediction: {predicted_class} - Confidence: {prediction[0][0]:.2f}')
    plt.show()

    return predicted_class, prediction[0][0]

# Assuming the user uploads an image and the path is stored in `user_uploaded_image_path`
user_uploaded_image_path = '/content/download (2).jpeg'
predicted_class, confidence = predict_disease(user_uploaded_image_path)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
